# Demo for paper "First Order Motion Model for Image Animation"

https://github.com/AliaksandrSiarohin/first-order-model/blob/master/demo.ipynb

**Clone repository**

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [2]:
cd first-order-model

/content/first-order-model


**Mount your Google drive folder on Colab**

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Add folder https://drive.google.com/drive/folders/1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw?usp=sharing  to your google drive.
Alternativelly you can use this mirror link https://drive.google.com/drive/folders/16inDpBRPT1UC0YMGMX3dKvRnOUsf5Dhn?usp=sharing**

**Load driving video and source image**

https://stackoverflow.com/questions/62759748/downloading-data-from-a-shared-google-drive-link-in-google-colab

In [4]:
%cd '/content/gdrive/My Drive/'

/content/gdrive/My Drive


In [5]:
!pip install kora
from kora import drive
drive.download_folder('1kZ1gCnpfU0BnpdU47pLM_TQ6RypDDqgw')

1TTBZhturn_wYMEZuiBsDS7lIfh0r0ukE 00.mp4 video/mp4 (1/41)
Download 100%.
1TjLR2-iFPzGP1o4ujq4tjaEnW6evd27H 01.png image/png (2/41)
Download 100%.
1b8dEqviM-UvMlKh2oKnWNMDGTsAjuAgW 02.png image/png (3/41)
Download 100%.
1BX2RmWsvU-YBDuVliOU54k-Jx8-Ib426 03.png image/png (4/41)
Download 100%.
1-3e_CgUnB7WUIdoSEGatv7klPfk8pRc0 04.mp4 video/mp4 (5/41)
Download 100%.
1BVcOJK5SVz18jsXudlVIxHvSQfHHwkN2 05.png image/png (6/41)
Download 100%.
1f4swpxjTCFgi19YWIT8Ca60J8XhWI6q1 06.png image/png (7/41)
Download 100%.
1NAGCeLzztOztwF0jPje6arXUgT57FQEd 07.mkv video/x-matroska (8/41)
Download 100%.
1SxuFS9VWPBL-_MPql69fwZc8A2LN8cSK 08.mp4 video/mp4 (9/41)
Download 100%.
1y-Azc67JYU6Ad7-sH8KcD7WGo9QgDQ1l 09.png image/png (10/41)
Download 100%.
1ddo9cCw2FqXBjJCkec_34P_Ene0n5llp 10-backward.mp4 video/mp4 (11/41)
Download 100%.
1S1e9mryzdbb_nC5tOZ6_UTVdID39TXVM 10.mp4 video/mp4 (12/41)
Download 100%.
1MaczVsmz6iGhm4zzEioFT7enYzztdLWs 11.png image/png (13/41)
Download 100%.
1QclQ5x_WQ7J_SyoHFow29e740YBPnv

'first-order-motion-model'

In [6]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/02.png')
reader = imageio.get_reader('/content/gdrive/My Drive/first-order-motion-model/04.mp4')


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

**Create a model and load checkpoints**

In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

ModuleNotFoundError: ignored

In [8]:
%cd "first-order-model/"

[Errno 2] No such file or directory: 'first-order-model/'
/content/gdrive/My Drive


In [9]:
%cd "/content/first-order-model/"

/content/first-order-model


In [10]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')

**Perform image animation**

In [11]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 26.50it/s]


**In the cell above we use relative keypoint displacement to animate the objects. We can use absolute coordinates instead,  but in this way all the object proporions will be inherited from the driving video. For example Putin haircut will be extended to match Trump haircut.**

In [12]:
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=False, adapt_movement_scale=True)
HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 211/211 [00:07<00:00, 27.07it/s]


## Running on your data

**First we need to crop a face from both source image and video, while simple graphic editor like paint can be used for cropping from image. Cropping from video is more complicated. You can use ffpmeg for this.**

In [13]:
!ffmpeg -i /content/gdrive/My\ Drive/first-order-motion-model/07.mkv -ss 00:08:57.50 -t 00:00:08 -filter:v "crop=600:600:760:50" -async 1 hinton.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

**Another posibility is to use some screen recording tool, or if you need to crop many images at ones use face detector(https://github.com/1adrianb/face-alignment) , see https://github.com/AliaksandrSiarohin/video-preprocessing for preprcessing of VoxCeleb.** 

In [14]:
source_image = imageio.imread('/content/gdrive/My Drive/first-order-motion-model/09.png')
driving_video = imageio.mimread('hinton.mp4', memtest=False)


#Resize image and video to 256x256

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 240/240 [00:09<00:00, 26.27it/s]
